In [27]:
import pandas as pd
from Bio.SeqIO.FastaIO import SimpleFastaParser

fasta = open("disease.fasta")
df_drugs = pd.read_csv('cann_mols.csv')  

fasta_list = list(SimpleFastaParser(fasta))

protein_df = pd.DataFrame(fasta_list, columns=['info', 'sequence'])

def string_splitter(string):
    string = string.split("HUMAN",1)[1]
    string = string.split("OS=Homo sapiens")
    
    return string

def info_parser(dfx):
    df = dfx.copy()
    df = df['info'].str.split('|', expand=True) # split on the "|" character
    df.columns = ['type', 'id', 'info'] # rename the three columns # rename the three columns

    return df

def info_parser_200k(dfx):
    df = dfx.copy()
    df = df['info'].str.split('|', expand=True) # split on the "|" character
    df.columns = ['type', 'id', 'info', 'none'] # rename the three columns # rename the three columns
    df.drop("none", axis=1, inplace=True) # drop info column

    return df

def info_pre_processed(dfx):
    df = dfx.copy() # create a copy of df
    list_of_info = list(dfx['info']) # convert column values to list
    list_of_info = [string_splitter(x) for x in list_of_info] # apply string_splitter function to list elements
    df['temp_col'] = list_of_info # create a temporary column from the processed list
    df.drop("info", axis=1, inplace=True) # drop info column
    split_df = pd.DataFrame(df['temp_col'].tolist(), columns=['Protein name', 'info']) # creating this dataframe to merge back onto processed dataframe
    df.drop("temp_col", axis=1, inplace=True) # dropping temp_col
    df = pd.concat([df, split_df], axis=1) # merging both dataframes

    return df
    
def info_processed(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df.columns = ['type', 'id', 'Protein name', 'drop', 'Species', 'Gene', 'PE', 'Mutation'] # rename columns
    df.drop("drop", axis=1, inplace=True) # drop empty column
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df.drop('Species', axis=1, inplace=True) # drop column
    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

def info_processed_200k(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df = df.drop(df.columns[[0, 3, 4, 6, 7, 8, 9, 10, 11, 12]],axis = 1)

    df.columns = ['id', 'Protein name', 'Gene'] # rename columns
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column

    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

def info_processed_disease(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df = df.drop(df.columns[[0, 3, 4, 6, 7, 8, 9]],axis = 1)

    df.columns = ['id', 'Protein name', 'Gene'] # rename columns
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column

    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

#clean_proteins = info_parser_200k(protein_df)
                 #pipe(info_parser).
                 #pipe(info_pre_processed).
                 #pipe(info_processed))
            
clean_proteins = (protein_df.
                 pipe(info_parser).
                 pipe(info_pre_processed).
                 pipe(info_processed_disease))



df_targets = clean_proteins[['Protein name','sequence']].copy() # making a copy of the clean dataframe.

target_name = df_targets['Protein name'].tolist()
target = df_targets.sequence.tolist()
drug_name = df_drugs.Name.tolist()
drug = df_drugs.SMILES.tolist()

df_targets.to_csv('disease_targets.csv')
clean_proteins.to_csv('clean_proteins_disease.csv')

,Name,SMILES
0,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
1,Cannflavin B,CC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C(C=C3)...
2,Cannflavin C,CC(=CCCC(=CCC1=C2C(=C(C=C1O)O)C(=O)C=C(O2)C3=C...
3,Canniprene,CC(=CCC1=C(C=CC(=C1O)OC)CCC2=CC(=CC(=C2)OC)O)C
4,Cannabistilbene I,CC(=CCC1=C(C=CC(=C1)CCC2=CC(=CC(=C2)OC)O)O)C
5,6-prenylnaringenin,CC(=CCC1=C(C2=C(C=C1O)OC(CC2=O)C3=CC=C(C=C3)O)O)C
6,Hesperetin,COC1=C(C=C(C=C1)C2CC(=O)C3=C(C=C(C=C3O2)O)O)O
7,Luteolin,C1=CC(=C(C=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O)O
8,Quercetin,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O


In [16]:
def info_processed_disease(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df = df.drop(df.columns[[0, 3, 4, 6, 7, 8, 9]],axis = 1)

    df.columns = ['id', 'Protein name', 'Gene'] # rename columns
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column

    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

#clean_proteins = info_parser_200k(protein_df)
                 #pipe(info_parser).
                 #pipe(info_pre_processed).
                 #pipe(info_processed))
            
clean_proteins = (protein_df.
                 pipe(info_parser).
                 pipe(info_pre_processed))

test = info_processed_200k(clean_proteins)

test

,id,Protein name,Gene,sequence
0,A0A024QZB8,Battenin,CLN3,MGGCAGSRRRFSDSEGEETVPEPRLPLLDHQGAHWKNAVGFWLLGL...
1,A0A218KGR2,Amyloid-beta A4 protein,APP,MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...
2,A0A5B9,T cell receptor beta constant 2,TRBC2,DLKNVFPPKVAVFEPSEAEISHTQKATLVCLATGFYPDHVELSWWV...
3,A0AV02,Solute carrier family 12 member 8,SLC12A8,MTQMSQVQELFHEAAQQDALAQPQPWWKTQLFMWEPVLFGTWDGVF...
4,A0PJY2,Fez family zinc finger protein 1,FEZF1,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...
...,...,...,...,...
6631,X5CF33,BTNL2,BTNL2,MVDFPGYNLSGAVASFLFILLTMKQSEDFRVIGPAHPILAGVGEDA...
6632,X5CF57,BRD2,BRD2,MASVPALQLTPANPPPPEVSNPKKPGRVTNQLQYLHKVVMKALWKH...
6633,X5CKA9,MHC class II antigen (Fragment),HLA-DPB1,MMVLQVSAAPRTVALTALLMVLLTSVVQGRATPENYVHQLRQECYA...
6634,X5D146,BTNL2,BTNL2,MVDFPGYNLSGAVASFLFILLTMKQSEDFRVIGPAHPILAGVGEDA...


,type,id,Protein name,0,1,2,3,4,5,6
0,tr,A0A024QZB8,Battenin,,OX=9606,GN=CLN3,PE=1,SV=1,None,None
1,tr,A0A218KGR2,Amyloid-beta A4 protein,,OX=9606,GN=APP,PE=3,SV=1,None,None
2,sp,A0A5B9,T cell receptor beta constant 2,,OX=9606,GN=TRBC2,PE=1,SV=2,None,None
3,sp,A0AV02,Solute carrier family 12 member 8,,OX=9606,GN=SLC12A8,PE=1,SV=4,None,None
4,sp,A0PJY2,Fez family zinc finger protein 1,,OX=9606,GN=FEZF1,PE=1,SV=1,None,None
...,...,...,...,...,...,...,...,...,...,...
6631,tr,X5CF33,BTNL2,,OX=9606,GN=BTNL2,PE=4,SV=1,None,None
6632,tr,X5CF57,BRD2,,OX=9606,GN=BRD2,PE=4,SV=1,None,None
6633,tr,X5CKA9,MHC class II antigen (Fragment),,OX=9606,GN=HLA-DPB1,PE=4,SV=1,None,None
6634,tr,X5D146,BTNL2,,OX=9606,GN=BTNL2,PE=4,SV=1,None,None


In [7]:
clean_proteins

,type,id,Protein name,info
0,tr,A0A024QZB8,Battenin,OX=9606 GN=CLN3 PE=1 SV=1
1,tr,A0A218KGR2,Amyloid-beta A4 protein,OX=9606 GN=APP PE=3 SV=1
2,sp,A0A5B9,T cell receptor beta constant 2,OX=9606 GN=TRBC2 PE=1 SV=2
3,sp,A0AV02,Solute carrier family 12 member 8,OX=9606 GN=SLC12A8 PE=1 SV=4
4,sp,A0PJY2,Fez family zinc finger protein 1,OX=9606 GN=FEZF1 PE=1 SV=1
...,...,...,...,...
6631,tr,X5CF33,BTNL2,OX=9606 GN=BTNL2 PE=4 SV=1
6632,tr,X5CF57,BRD2,OX=9606 GN=BRD2 PE=4 SV=1
6633,tr,X5CKA9,MHC class II antigen (Fragment),OX=9606 GN=HLA-DPB1 PE=4 SV=1
6634,tr,X5D146,BTNL2,OX=9606 GN=BTNL2 PE=4 SV=1


In [ ]:
df_targets['drug_name'] = df_drugs['Name'][0] # adding a column with a constant value of "cannaflavin a".
df_targets['SMILES'] = df_drugs['SMILES'][0] # adding a column with a constant value of "cannaflavin a" SMILES string.

df_targets_2 = df_targets.copy()
df_targets_2['drug_name'] = df_drugs['Name'][1]
df_targets_2['SMILES'] = df_drugs['SMILES'][1]

df_targets_3 = df_targets.copy()
df_targets_3['drug_name'] = df_drugs['Name'][2]
df_targets_3['SMILES'] = df_drugs['SMILES'][2]

df_targets_4 = df_targets.copy()
df_targets_4['drug_name'] = df_drugs['Name'][3]
df_targets_4['SMILES'] = df_drugs['SMILES'][3]

df_targets_5 = df_targets.copy()
df_targets_5['drug_name'] = df_drugs['Name'][4]
df_targets_5['SMILES'] = df_drugs['SMILES'][4]

df_targets_6 = df_targets.copy()
df_targets_6['drug_name'] = df_drugs['Name'][5]
df_targets_6['SMILES'] = df_drugs['SMILES'][5]

df_targets_7 = df_targets.copy()
df_targets_7['drug_name'] = df_drugs['Name'][6]
df_targets_7['SMILES'] = df_drugs['SMILES'][6]

df_targets_8 = df_targets.copy()
df_targets_8['drug_name'] = df_drugs['Name'][7]
df_targets_8['SMILES'] = df_drugs['SMILES'][7]

df_targets_9 = df_targets.copy()
df_targets_9['drug_name'] = df_drugs['Name'][8]
df_targets_9['SMILES'] = df_drugs['SMILES'][8]

targets = pd.concat([df_targets, 
                  df_targets_2, 
                  df_targets_3, 
                  df_targets_4, 
                  df_targets_5, 
                  df_targets_6, 
                  df_targets_7, 
                  df_targets_8, 
                  df_targets_9], axis=0)

In [31]:
len(target_name)

1844154

In [28]:
clean_proteins
df_targets

,Protein name,sequence
0,"T-box 2, isoform CRA_a",MREPALAASAMAYHPFHAPRPADFPMSAFLAAAQPSFFPALALPPG...
1,Receptor protein-tyrosine kinase,MELQAARACFALLWGCALAAAAAAQGKEVVLLDFAAAGGELGWLTH...
2,Battenin,MGGCAGSRRRFSDSEGEETVPEPRLPLLDHQGAHWKNAVGFWLLGL...
3,Sirtuin (Silent mating type information regul...,MADEAALALQPGGSPSAAGADREAASSPAGEPLRKRPRRDGPGLER...
4,Phospholipid phosphatase 1,MFDKTRLPYVALDVLCVLLAGLPFAILTSRHTPFQRGVFCNDESIK...
...,...,...
204901,Probable ATP-dependent RNA helicase DDX5,MSGYSSDRDRGRDRGFGAPRFGGSRAGPLSGKKFGNPGEKLVKKKW...
204902,ELKS/Rab6-interacting/CAST family member 1,MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSV...
204903,Voltage-dependent calcium channel subunit alp...,MKLEFLQRKFWAATRQCSTVDGPCTQSCEDSDLDCFVIDNNGFILI...
204904,ELKS/Rab6-interacting/CAST family member 1,MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSV...


In [ ]:
import pandas as pd
from Bio.SeqIO.FastaIO import SimpleFastaParser

fasta = open("uniprot_200k.fasta")
df_drugs = pd.read_csv('cann_mols.csv')  

fasta_list = list(SimpleFastaParser(fasta))

protein_df = pd.DataFrame(fasta_list, columns=['info', 'sequence'])

def string_splitter(string):
    string = string.split("HUMAN",1)[1]
    string = string.split("OS=Homo sapiens")
    
    return string

def info_parser(dfx):
    df = dfx.copy()
    df = df['info'].str.split('|', expand=True) # split on the "|" character
    df.columns = ['type', 'id', 'info'] # rename the three columns # rename the three columns

    return df



def info_parser_200k(dfx):
    df = dfx.copy()
    df = df['info'].str.split('|', expand=True) # split on the "|" character
    df.columns = ['type', 'id', 'info', 'none'] # rename the three columns # rename the three columns
    df.drop("none", axis=1, inplace=True) # drop info column

    return df

def info_pre_processed(dfx):
    df = dfx.copy() # create a copy of df
    list_of_info = list(dfx['info']) # convert column values to list
    list_of_info = [string_splitter(x) for x in list_of_info] # apply string_splitter function to list elements
    df['temp_col'] = list_of_info # create a temporary column from the processed list
    df.drop("info", axis=1, inplace=True) # drop info column
    split_df = pd.DataFrame(df['temp_col'].tolist(), columns=['Protein name', 'info']) # creating this dataframe to merge back onto processed dataframe
    df.drop("temp_col", axis=1, inplace=True) # dropping temp_col
    df = pd.concat([df, split_df], axis=1) # merging both dataframes

    return df
    
def info_processed(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df.columns = ['type', 'id', 'Protein name', 'drop', 'Species', 'Gene', 'PE', 'Mutation'] # rename columns
    df.drop("drop", axis=1, inplace=True) # drop empty column
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df.drop('Species', axis=1, inplace=True) # drop column
    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

def info_processed_200k(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df = df.drop(df.columns[[0, 3, 4, 6, 7, 8, 9, 10, 11, 12]],axis = 1)

    df.columns = ['id', 'Protein name', 'Gene'] # rename columns
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column

    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

#clean_proteins = info_parser_200k(protein_df)
                 #pipe(info_parser).
                 #pipe(info_pre_processed).
                 #pipe(info_processed))
            
clean_proteins = (protein_df.
                 pipe(info_parser_200k).
                 pipe(info_pre_processed).
                 pipe(info_processed_200k))

df_targets = clean_proteins[['Protein name','sequence']].copy() # making a copy of the clean dataframe.

df_targets['drug_name'] = df_drugs['Name'][0] # adding a column with a constant value of "cannaflavin a".
df_targets['SMILES'] = df_drugs['SMILES'][0] # adding a column with a constant value of "cannaflavin a" SMILES string.

df_targets_2 = df_targets.copy()
df_targets_2['drug_name'] = df_drugs['Name'][1]
df_targets_2['SMILES'] = df_drugs['SMILES'][1]

df_targets_3 = df_targets.copy()
df_targets_3['drug_name'] = df_drugs['Name'][2]
df_targets_3['SMILES'] = df_drugs['SMILES'][2]

df_targets_4 = df_targets.copy()
df_targets_4['drug_name'] = df_drugs['Name'][3]
df_targets_4['SMILES'] = df_drugs['SMILES'][3]

df_targets_5 = df_targets.copy()
df_targets_5['drug_name'] = df_drugs['Name'][4]
df_targets_5['SMILES'] = df_drugs['SMILES'][4]

df_targets_6 = df_targets.copy()
df_targets_6['drug_name'] = df_drugs['Name'][5]
df_targets_6['SMILES'] = df_drugs['SMILES'][5]

df_targets_7 = df_targets.copy()
df_targets_7['drug_name'] = df_drugs['Name'][6]
df_targets_7['SMILES'] = df_drugs['SMILES'][6]

df_targets_8 = df_targets.copy()
df_targets_8['drug_name'] = df_drugs['Name'][7]
df_targets_8['SMILES'] = df_drugs['SMILES'][7]

df_targets_9 = df_targets.copy()
df_targets_9['drug_name'] = df_drugs['Name'][8]
df_targets_9['SMILES'] = df_drugs['SMILES'][8]

targets = pd.concat([df_targets, 
                  df_targets_2, 
                  df_targets_3, 
                  df_targets_4, 
                  df_targets_5, 
                  df_targets_6, 
                  df_targets_7, 
                  df_targets_8, 
                  df_targets_9], axis=0)

target_name = targets['Protein name'].tolist()
target = targets.sequence.tolist()
drug_name = targets.drug_name.tolist()
drug = targets.SMILES.tolist()

targets = pd.read_csv('targets_pre_screen.csv')
clean_proteins.to_csv('clean_proteins.csv')

In [ ]:
import pandas as pd
from DeepPurpose import DTI as models
targets = pd.read_csv('targets_pre_screen.csv')

target_name = targets['Protein name'].tolist()
target = targets.sequence.tolist()
drug_name = targets.drug_name.tolist()
drug = targets.SMILES.tolist()

# Virtual screening using the trained model or pre-trained model 

net = models.model_pretrained(model = 'Morgan_AAC_BindingDB_IC50')

_ = models.virtual_screening(drug, target, net, drug_name, target_name)

In [35]:
from Bio.SeqIO.FastaIO import SimpleFastaParser

fasta = open("uniprot_200k.fasta")
df_drugs = pd.read_csv('cann_mols.csv')  

fasta_list = list(SimpleFastaParser(fasta))

protein_df = pd.DataFrame(fasta_list, columns=['info', 'sequence'])

def string_splitter(string):
    string = string.split("HUMAN",1)[1]
    string = string.split("OS=Homo sapiens")
    
    return string

def info_parser(dfx):
    df = dfx.copy()
    df = df['info'].str.split('|', expand=True) # split on the "|" character
    df.columns = ['type', 'id', 'info'] # rename the three columns # rename the three columns

    return df



def info_parser_200k(dfx):
    df = dfx.copy()
    df = df['info'].str.split('|', expand=True) # split on the "|" character
    df.columns = ['type', 'id', 'info', 'none'] # rename the three columns # rename the three columns
    df.drop("none", axis=1, inplace=True) # drop info column

    return df

def info_pre_processed(dfx):
    df = dfx.copy() # create a copy of df
    list_of_info = list(dfx['info']) # convert column values to list
    list_of_info = [string_splitter(x) for x in list_of_info] # apply string_splitter function to list elements
    df['temp_col'] = list_of_info # create a temporary column from the processed list
    df.drop("info", axis=1, inplace=True) # drop info column
    split_df = pd.DataFrame(df['temp_col'].tolist(), columns=['Protein name', 'info']) # creating this dataframe to merge back onto processed dataframe
    df.drop("temp_col", axis=1, inplace=True) # dropping temp_col
    df = pd.concat([df, split_df], axis=1) # merging both dataframes

    return df
    
def info_processed(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df.columns = ['type', 'id', 'Protein name', 'drop', 'Species', 'Gene', 'PE', 'Mutation'] # rename columns
    df.drop("drop", axis=1, inplace=True) # drop empty column
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df.drop('Species', axis=1, inplace=True) # drop column
    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

def info_processed_200k(dfx):
    df = dfx.copy() # create a copy of the dataframe
    split_df = df['info'].str.split(' ', expand=True) # creating a seperate dataframe to merge with
    df = pd.concat([df, split_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column
    df = df.drop(df.columns[[0, 3, 4, 6, 7, 8, 9, 10, 11, 12]],axis = 1)

    df.columns = ['id', 'Protein name', 'Gene'] # rename columns
    df = pd.concat([df, protein_df], axis=1) # merge dataframes
    df.drop("info", axis=1, inplace=True) # drop info column

    df['Gene'] = df['Gene'].str[3:] # strip first 3 characters
    
    return df

#clean_proteins = info_parser_200k(protein_df)
                 #pipe(info_parser).
                 #pipe(info_pre_processed).
                 #pipe(info_processed))
            
clean_proteins = (protein_df.
                 pipe(info_parser_200k).
                 pipe(info_pre_processed).
                 pipe(info_processed_200k))

targets = clean_proteins[['Protein name','sequence']].copy() # making a copy of the clean dataframe.

targets['drug_name'] = df_drugs['Name'][0] # adding a column with a constant value of "cannaflavin a".
targets['SMILES'] = df_drugs['SMILES'][0] # adding a column with a constant value of "cannaflavin a" SMILES string.

target_name = targets['Protein name'].tolist()
target = targets.sequence.tolist()
drug_name = targets.drug_name.tolist()
drug = targets.SMILES.tolist()

targets.to_csv('targets_pre_screen_cann_a.csv')
#clean_proteins.to_csv('clean_proteins.csv')

,Protein name,sequence,drug_name,SMILES
0,"T-box 2, isoform CRA_a",MREPALAASAMAYHPFHAPRPADFPMSAFLAAAQPSFFPALALPPG...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
1,Receptor protein-tyrosine kinase,MELQAARACFALLWGCALAAAAAAQGKEVVLLDFAAAGGELGWLTH...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
2,Battenin,MGGCAGSRRRFSDSEGEETVPEPRLPLLDHQGAHWKNAVGFWLLGL...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
3,Sirtuin (Silent mating type information regul...,MADEAALALQPGGSPSAAGADREAASSPAGEPLRKRPRRDGPGLER...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
4,Phospholipid phosphatase 1,MFDKTRLPYVALDVLCVLLAGLPFAILTSRHTPFQRGVFCNDESIK...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
...,...,...,...,...
204901,Probable ATP-dependent RNA helicase DDX5,MSGYSSDRDRGRDRGFGAPRFGGSRAGPLSGKKFGNPGEKLVKKKW...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
204902,ELKS/Rab6-interacting/CAST family member 1,MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSV...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
204903,Voltage-dependent calcium channel subunit alp...,MKLEFLQRKFWAATRQCSTVDGPCTQSCEDSDLDCFVIDNNGFILI...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
204904,ELKS/Rab6-interacting/CAST family member 1,MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSV...,Cannflavin A,CC(=CCCC(=CCC1=C(C2=C(C=C1O)OC(=CC2=O)C3=CC(=C...
